# Chains: 链

In [7]:
import os
import warnings
from IPython.display import Markdown
from setenv import APIKeyManager

# 忽略警告
warnings.filterwarnings('ignore')
# 设置API密钥
key_manager = APIKeyManager().setup_api_key()

# 安装 
# !pip install duckduckgo_search

# 查看Python版本
!python -V
# 查看安装的库
!pip list | grep -E 'langchain|openai|llm|tiktoken|chromadb|cryptography|duck'

Python 3.9.21
cryptography              44.0.2
duckduckgo_search         7.5.3
langchain                 0.3.21
langchain-community       0.3.20
langchain-core            0.3.47
langchain-openai          0.3.9
langchain-text-splitters  0.3.7
openai                    1.68.2
tiktoken                  0.9.0


## 1. 基础链

### 1.1 LLMChain

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 创建 LLM
llm = ChatOpenAI(model="deepseek-chat", temperature=0.7)

# 创建提示词模板
prompt = PromptTemplate(
    input_variables=["question"],
    template="请回答以下问题：{question}"
)

# 创建 LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# 运行链
response = chain.run("什么是人工智能？")

In [8]:
Markdown(response)

人工智能（Artificial Intelligence，简称 AI）是指通过计算机系统模拟、扩展和增强人类智能的技术和科学。它旨在使机器能够执行通常需要人类智能的任务，例如学习、推理、问题解决、感知、语言理解和决策等。

人工智能可以分为以下几类：

1. **弱人工智能（Narrow AI）**：专注于执行特定任务的人工智能，例如语音识别、图像识别或推荐系统。这类 AI 在特定领域表现出色，但不具备通用智能。

2. **强人工智能（General AI）**：具备与人类相当的通用智能，能够理解、学习和执行任何智力任务。目前，强人工智能尚未实现，仍处于理论阶段。

3. **超级人工智能（Superintelligent AI）**：指在几乎所有领域都超越人类智能的 AI。这是未来可能的发展方向，但也引发了伦理和安全方面的讨论。

人工智能的核心技术包括：
- **机器学习（Machine Learning）**：通过数据训练模型，使机器能够从经验中学习并改进性能。
- **深度学习（Deep Learning）**：一种基于神经网络的机器学习方法，适用于处理复杂的数据（如图像、语音和文本）。
- **自然语言处理（NLP）**：使机器能够理解、生成和处理人类语言。
- **计算机视觉（Computer Vision）**：使机器能够“看”并理解图像和视频内容。
- **强化学习（Reinforcement Learning）**：通过试错和奖励机制训练机器完成特定任务。

人工智能已广泛应用于各个领域，如医疗、金融、教育、交通和娱乐等，正在深刻改变我们的生活和工作方式。

### 1.2 SequentialChain

In [14]:
from langchain.chains import SequentialChain

# 创建提示词模板
prompt1 = PromptTemplate(
    input_variables=["input"],
    template="请回答以下问题：{input}"
)
prompt2 = PromptTemplate(
    input_variables=["output1"],
    template="请用简洁的语言总结以下内容：{output1}"
)

# 创建多个链
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="output1")
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="output2")

# 创建顺序链
sequential_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["input"],
    output_variables=["output1", "output2"]
)

# 运行顺序链
response = sequential_chain({"input": "什么是人工智能？"})
print("第一个链的输出：")
Markdown(response["output1"])
print("最终总结：")
Markdown(response["output2"])

第一个链的输出：
最终总结：


人工智能（AI）是通过计算机系统模拟人类智能，使其能够执行学习、推理、问题解决等任务。AI分为三类：**弱人工智能**（专注于特定任务，如语音助手）、**强人工智能**（具备通用智能，尚未实现）和**超级人工智能**（超越人类智能，仅存在于理论中）。AI的实现依赖于机器学习、深度学习、自然语言处理、计算机视觉和强化学习等技术，广泛应用于医疗、金融、交通等领域，正在深刻改变生活和工作方式。

## 2. 高级链

### 2.1 AgentChain

In [10]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI

# 大模型
llm = ChatOpenAI(temperature=0.8, model='deepseek-chat')

# 网络检索
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="用于搜索网络信息"
    )
]

# 初始化代理
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description", 
    verbose=True
)

# 运行代理
try:
    agent.run("最新的AI技术发展是什么？")
except Exception as e:
    print(f"搜索过程中发生错误: {e}")
    print("请尝试使用其他搜索工具或稍后再试")



> Entering new AgentExecutor chain...
为了回答关于最新AI技术发展的问题，我需要搜索最新的AI技术动态和趋势。  
Action: Search  
Action Input: "最新AI技术发展 2023"  
搜索过程中发生错误: https://lite.duckduckgo.com/lite/ RuntimeError: error sending request for url (https://lite.duckduckgo.com/lite/): operation timed out

Caused by:
    operation timed out
请尝试使用其他搜索工具或稍后再试


### 2.2 MemoryChain 

In [13]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


# 创建模型 - 使用DeepSeek的聊天模型
llm = ChatOpenAI(model='deepseek-chat')

# 提示词模板 - 定义输入变量和模板结构
# 包含两个变量：chat_history(对话历史)和question(当前问题)
prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template="""基于以下对话历史和问题，请给出合适的回答：

对话历史：
{chat_history}

当前问题：{question}

回答："""
)

# 创建记忆组件 - ConversationBufferMemory用于存储对话历史
# memory_key指定在提示模板中使用的变量名
# return_messages=True表示以消息对象形式返回历史，而不是纯文本
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 创建带记忆的链 - 将LLM、提示模板和记忆组件组合成一个链
# verbose=True启用详细输出，可以看到链的执行过程
# 这个链能够记住之前的对话内容，实现连续对话的能力
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)

In [14]:
# 第一次对话
response1 = chain.run("你好，请介绍一下自己")
print(f"第一次回答: {response1}\n")

# 第二次对话 - 测试记忆功能
response2 = chain.run("你刚才说了什么？")
print(f"第二次回答: {response2}\n")

# 第三次对话 - 提出新问题
response3 = chain.run("你能解释一下什么是人工智能吗？")
print(f"第三次回答: {response3}\n")

# 第四次对话 - 引用之前的内容
response4 = chain.run("基于你刚才对人工智能的解释，它与机器学习有什么关系？")
print(f"第四次回答: {response4}\n")



> Entering new LLMChain chain...
Prompt after formatting:
基于以下对话历史和问题，请给出合适的回答：

对话历史：
[]

当前问题：你好，请介绍一下自己

回答：

> Finished chain.
第一次回答: 你好！我是DeepSeek Chat，一个由深度求索公司开发的智能AI助手。我可以帮助你解答各种问题，包括学习、工作、生活、科技、娱乐等多个领域。我具备强大的自然语言处理能力，能够理解并生成高质量的文本内容，还能处理上传的文档（如PDF、Word、Excel等）来提取和分析信息。  

我的知识截至2024年7月，并且是**完全免费的**！无论是查找资料、写作辅助、代码编写，还是日常闲聊，我都乐意为你提供帮助。如果有任何问题，尽管问我吧！😊



> Entering new LLMChain chain...
Prompt after formatting:
基于以下对话历史和问题，请给出合适的回答：

对话历史：
[HumanMessage(content='你好，请介绍一下自己', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！我是DeepSeek Chat，一个由深度求索公司开发的智能AI助手。我可以帮助你解答各种问题，包括学习、工作、生活、科技、娱乐等多个领域。我具备强大的自然语言处理能力，能够理解并生成高质量的文本内容，还能处理上传的文档（如PDF、Word、Excel等）来提取和分析信息。  \n\n我的知识截至2024年7月，并且是**完全免费的**！无论是查找资料、写作辅助、代码编写，还是日常闲聊，我都乐意为你提供帮助。如果有任何问题，尽管问我吧！😊', additional_kwargs={}, response_metadata={})]

当前问题：你刚才说了什么？

回答：

> Finished chain.
第二次回答: 我刚才的自我介绍是：  
**"你好！我是DeepSeek Chat，由深度求索公司开发的智能AI助手。我能解答学习、工作、生活等多领域问题，支持文档分析（PDF/Word等），知识截至2024年7月，且完全免费。需要任何帮助随时问我！😊"**  

需要更